# Fine-tuning KB-ALBERT with `Transformers` PyTorch ver.

Hugging Face의 Transformers는 미세조정(fine-tuning)을 위해 필요한 API들을 간단하게 사용할 수 있는 형태로 제공합니다.
기초적인 파이썬 프로그래밍과 기계학습 모델의 학습 방법만 알고있다면, 자연어처리(NLP)를 전공하지 않은 일반 사용자도 쉽게 최신 인공지능 언어모델을 불러와 사용할 수 있습니다.

그리고 특정 목적을 위해 미세조정된 기계학습 모델을 새로운 데이터에서 쉽게 예측해 볼 수 있도록 Inference Pipeline도 API로 제공하고 있습니다. 그래서 누구나 쉽게 자신이 학습한 모델을 PyTorch나 TensorFlow의 naive programming을 하지 않아도 쉽게 모델을 학습하고 학습된 모델을 배포할 수 있습니다.


<br>

## Objective
- Hugging Face's Transformers를 활용한 fine-tuning

<br>

---

<br>

## 네이버 영화리뷰 감성분석 (Naver Movie Review Sentiment Analysis)

Naver sentiment movie corpus([link](https://github.com/e9t/nsmc))는 한국어 영화 리뷰 데이터로 리뷰 내용의 긍정과 부정이 라벨링 된 데이터입니다. 15만 개의 학습 데이터와 5만 개의 테스트 데이터로 나누어져 있습니다. 140개 이하의 짧은 문장으로 되어 있고, 쉽게 접근하여 사용해볼 수 있는 데이터입니다.

<br>

## Contents

1. 데이터 준비 & 필요 소스코드 다운로드
2. Tokenizer를 활용한 학습용 데이터셋 생성
3. 학습 하이퍼파라미터 설정
4. Head를 활용한 모델 Fine-tuning
5. Inference를 위한 Pipeline 생성

<br>
<br>

# 1. 데이터 및 필요 소스코드 등 준비

예제를 수행하기 위해 필요한 데이터와 소스코드를 다운로드 해야 합니다.
그리고 아쉽게도 KB-ALBERT는 내부 라이선스 이슈로 Transformers Model Hub에 업로드되어 있지 않아 별도의 과정을 통해 요청하신 후에 다운로드 받아 사용할 수 있습니다. 요청 방법은 [링크](https://github.com/KB-BANK-AI/KB-ALBERT-KO/kb-albert-char)를 참고해주세요.

> 참고: 본 예제는 음절단위 모델을 기준으로 작성되었습니다.

<br>

실행 내용
1. Download source code for Custom Tokenizer
2. Install Transformers library
3. Download NSMC dataset

**원활한 모델 학습을 위해 GPU 환경에서 테스트하시기를 권장드립니다.<br> 위 메뉴창에서 "런타임" > "런타임 유형변경" > "하드웨어가속기 GPU 선택" > "저장" 으로 환경을 변경하실 수 있습니다.**

In [1]:
# Download source codes
!git clone https://github.com/sackoh/pycon-korea-2020-kb-albert.git
%cd pycon-korea-2020-kb-albert/

# Install transformers
%pip install -q transformers

# Download NSMC dataset
!wget -q https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
!wget -q https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

Cloning into 'pycon-korea-2020-kb-albert'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 41 (delta 19), reused 19 (delta 8), pack-reused 0
Unpacking objects: 100% (41/41), 1.66 MiB | 6.43 MiB/s, done.
/content/pycon-korea-2020-kb-albert
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 14.8 MB/s eta 0:00:00


<br>
<br>

## Upload pretrained model and configuration files

다운로드 받은 파일을 **Google Colab**에서 사용하기 위해서는 파일 업로드가 필요합니다. 아래의 코드를 실행해주세요. 아래를 실행하면 파일 업로드를 위한 팝업창이 열립니다. 다음의 4가지 파일들을 *반드시* 선택하여 업로드를 진행해주세요.

- config.json
- pytorch_model.bin
- tokenizer_config.json
- vocab.txt

업로드에 다소 시간이 걸릴 수 있습니다.

In [2]:
from google.colab import files
uploaded = files.upload()

%mkdir kb-albert-char
%mv config.json pytorch_model.bin tokenizer_config.json vocab.txt ./kb-albert-char

Saving config.json to config.json
Saving pytorch_model.bin to pytorch_model.bin
Saving tokenizer_config.json to tokenizer_config.json
Saving vocab.txt to vocab.txt


<br>
<br>

## Load train & test data

NSMC 데이터를 불러옵니다. train data와 test data는 각각 리뷰 텍스트와 리뷰의 긍부정 라벨로 구성되어 있습니다.

| text | label |
| ---  | ---   |
| 아 더빙.. 진짜 짜증나네요 목소리 | 0 |
| 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 | 1 |
| 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ... | 1 |

In [3]:
import csv
from pathlib import Path

def read_nsmc_data(file_path):
    texts = []
    labels = []
    with open(file_path, "r", encoding="utf-8") as r:
        reader = csv.reader(r, delimiter="\t")
        next(reader, None)
        for line in reader:
            texts.append(line[1])
            labels.append(int(line[2]))
    return texts, labels

data_dir = Path('./')
train_texts, train_labels = read_nsmc_data(data_dir/'ratings_train.txt')
test_texts, test_labels = read_nsmc_data(data_dir/'ratings_test.txt')

<br>
<br>

# 2. Tokenizer를 활용한 학습용 데이터셋 생성



## Custom Tokenizer 불러오기

본 예제는 음절단위(Character-level) 모델을 사용하기 때문에 한국어를 위한 음절단위 Tokenizer를 사용해야 합니다. `Transformers`에서 공식 Tokenizer로 등록이 안됐기 때문에 이전 단계에서 다운로드한 소스코드에서 Custom Tokenizer를 불러옵니다.

만약 공식 Hub에 등록된 토크나이저를 사용한다면 다음과 같이 불러오면 됩니다.
```python
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
```

In [4]:
from tokenization_kbalbert import KbAlbertCharTokenizer

tokenizer = KbAlbertCharTokenizer.from_pretrained('kb-albert-char')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KbAlbertCharTokenizer'.


<br>
<br>

## Tokenizer를 통해 Raw text를 전처리

불러온 tokenizer를 활용해 input 데이터를 전처리합니다. 텍스트를 sparse indices 형태로 변환하는 것이 주요 처리 내용입니다.

In [5]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

<br>
<br>

## PyTorch Dataset으로 학습용 데이터셋 생성

전처리된 데이터를 파이토치의 `Dataset` 으로 만들어줍니다. `Dataset`과 관련된 자세한 내용은 [링크](https://pytorch.org/docs/stable/data.html)를 참조해주세요.

In [6]:
import torch

class NSMCDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = NSMCDataset(train_encodings, train_labels)
test_dataset = NSMCDataset(test_encodings, test_labels)

<br>
<br>

## 3. 학습 하이퍼파라미터 설정

`Transformers`에서 제공하는 `TrainingArguments`에 학습 하이퍼파라미터를 설정해줍니다.

설정하지 않은 하이퍼파라미터들은 default 값을 사용합니다.

하이퍼파라미터 설정을 위한 상세 내용은 [링크](https://huggingface.co/transformers/main_classes/trainer.html?highlight=trainingargument#transformers.TrainingArguments)를 참조해주시기 바랍니다.

In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2.0,           # 에폭
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=2000
)

<br>
<br>

# 4. Head를 활용한 모델 Fine-tuning

네이버 영화리뷰 감성분석을 위해 텍스트(sequence of words)에서 긍부정(1/0)을 예측하는 모델을 학습합니다. Sequence의 분류문제(Classification) 미세조정을 위한 Head인 `XXXForSequenceClassification`을 불러옵니다. 본 예제는 ALBERT 언어모델을 사용하기 때문에 `AlbertForSequenceClassification` Head class를 불러옵니다.

## Fine-tuning을 위한 Head에 사전학습한 언어모델 불러오기

Head의 `from_pretrained` 메소드를 통해 사전학습한 KB-ALBERT 언어모델을 함께 불러옵니다. 이때 인자값으로 모델이 위치한 디렉토리(폴더) 경로를 넘겨줍니다.

> Head는 Pretrained Language Model과 Output Layer로 구성되어 있습니다.

미세조정을 통해 Language Model과 Output Layer의 weight가 함께 조정됩니다.

In [8]:
from transformers import AlbertForSequenceClassification

model = AlbertForSequenceClassification.from_pretrained('kb-albert-char')

Some weights of the model checkpoint at kb-albert-char were not used when initializing AlbertForSequenceClassification: ['predictions.dense.weight', 'sop_classifier.classifier.weight', 'predictions.dense.bias', 'predictions.LayerNorm.bias', 'sop_classifier.classifier.bias', 'predictions.decoder.bias', 'predictions.bias', 'predictions.decoder.weight', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at kb-albert-char and a

<br>
<br>

## Trainer를 통해 fine-tuning 수행

그 다음은 학습을 위해 `Trainer` class를 불러옵니다. Head와 학습 하이퍼파라미터 그리고 앞에서 생성한 학습용 데이터셋을 인자로 넘긴 후에 `train()` 메소드로 학습을 시작합니다.


In [9]:
from transformers import Trainer
        
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 150000
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 9376
  Number of trainable parameters = 9074818


Step,Training Loss
2000,0.360200
4000,0.291100
6000,0.232300
8000,0.201900


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2500
Configuration saved in ./results/checkpoint-2500/config.json
Model weights saved in ./results/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-3000
Configuration saved in ./results/checkpoint-3

TrainOutput(global_step=9376, training_loss=0.25957037238944514, metrics={'train_runtime': 5626.2018, 'train_samples_per_second': 53.322, 'train_steps_per_second': 1.666, 'total_flos': 1792357632000000.0, 'train_loss': 0.25957037238944514, 'epoch': 2.0})

<br>
<br>

## (Optional) Test data에 fine-tuned 모델 성능 평가하기

모델 성능 평가를 위한 test dataset과 custom function을 trainer에게 전달하면 평가 결과를 반환합니다.

In [10]:
import numpy as np
from transformers import EvalPrediction

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return {
            'acc': (preds == p.label_ids).mean()
        }
trainer.compute_metrics = compute_metrics
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples = 50000
  Batch size = 64


{'eval_loss': 0.26203206181526184,
 'eval_acc': 0.89728,
 'eval_runtime': 360.0375,
 'eval_samples_per_second': 138.874,
 'eval_steps_per_second': 2.172,
 'epoch': 2.0}

<br>
<br>

# 5. Inference를 위한 Pipeline 생성

`Transformers`는 미세조정된 모델에 raw text를 인풋으로 넣었을 때 데이터 전처리와 모델 추론 과정을 `pipeline` class를 통해 api 형태로 쉽게 개발할 수 있도록 하였습니다.

pipeline에는 사전 정의된 task 유형과 미세조정된 모델, tokenizer 그리고 딥러닝 프레임워크 유형을 인자로 전달합니다. (PyTorch는 'pt', TensorFlow는 'tf')

생성된 pipeline 인스턴스에 raw text를 넣어주면 예측 결과와 예측 결과에 대한 confidence 값이 반환

In [35]:
from transformers import pipeline

nsmc_classifier = pipeline('sentiment-analysis', 
                           model=model.cpu(), tokenizer=tokenizer, framework='pt')
id2label = {"LABEL_0": "negative", "LABEL_1": "positive"}

reviews= [
          "하 너무 재밌었다 아이언맨 짱"
]

results = nsmc_classifier(reviews)
for idx, result in enumerate(results):
    print(reviews[idx])
    for k, v in result.items():
        print(f" >> {k} : {id2label[v] if k == 'label' else v}")
    print()

하 너무 재밌었다 아이언맨 짱
 >> label : positive
 >> score : 0.9786463379859924



pipeline이나 미세조정된 모델은 `save_pretrained`를 통해 지정된 경로에 저장할 수 있습니다. 다시 사용해야 할 때는 같은 경로명을 `from_pretrained`에 넣어주어 쉽게 불러올 수 있습니다.

In [26]:
nsmc_classifier.save_pretrained('./pipeline')

Configuration saved in ./pipeline/config.json
Model weights saved in ./pipeline/pytorch_model.bin
tokenizer config file saved in ./pipeline/tokenizer_config.json
Special tokens file saved in ./pipeline/special_tokens_map.json


TypeError: ignored

In [ ]:
# 현재 버전
# tokenizers                    0.13.2
# transformers                  4.26.0

# tensorflow                    2.9.2

In [21]:
!pip list

Package                       Version
----------------------------- ----------------------
absl-py                       1.3.0
aeppl                         0.0.33
aesara                        2.7.9
aiohttp                       3.8.3
aiosignal                     1.3.1
alabaster                     0.7.12
albumentations                1.2.1
altair                        4.2.0
appdirs                       1.4.4
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.2.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.11.0
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.1
blis                          0.7.9
bokeh                         2.3.3
branca